# Trabalho 1

11218813 Bruna Magrini da Cruz<br>
11218809 Flavio Salles<br>
11219004 Yasmin Osajima de Araujo

# Bibliotecas

In [1]:
%pip install glfw


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math


# Janela

In [3]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(1020, 900, "Trabalho 1", None, None)
glfw.make_context_current(window)


# GLSL

### `GLSL` (*OpenGL Shading Language*)
Aqui veremos nosso primeiro código `GLSL`.

É uma linguagem de *shading* de alto nível baseada na linguagem de programação `C`.

Estamos escrevendo código `GLSL` como *strings* de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Aprenderemos `GLSL` conforme a necessidade do curso. Usaremos uma versão do `GLSL` mais antiga, compatível com muitos dispositivos.
### `GLSL` para *Vertex Shader*
No Pipeline programável, podemos interagir com *Vertex Shaders*.

No código abaixo, estamos fazendo o seguinte:
* Definindo uma variável chamada `position` do tipo `vec2`.
* Definindo uma variável chamada `mat_transformation` do tipo `mat4` (matriz $4\times4$).
* Usamos `vec2`, pois nosso programa (na CPU) enviará apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (`vec3`) e até mesmo quatro coordenadas (`vec4`).
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `gl_Position` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a posição de um vértice. Observe que todo vértice tem $4$ coordenadas, por isso combinamos nossa variável `vec2` com uma variável `vec4`. Além disso, modificamos nosso vetor com base em uma matriz de transformação, conforme estudado na Aula 4.

In [4]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """


### `GLSL` para *Fragment Shader*
No Pipeline programável, podemos interagir com *Fragment Shaders*.

No código abaixo, estamos fazendo o seguinte:
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `gl_FragColor` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a cor de um fragmento.

### Possibilitando modificar a cor
Nos exemplos anteriores, a variável `gl_FragColor` estava definida de forma fixa (com cor $R=0$, $G=0$, $B=0$).

Agora, criaremos uma variável do tipo `uniform`, de quatro posições (`vec4`), para receber o dado de cor do nosso programa rodando em CPU.

In [5]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """


# Compilação e Linkagem

### Requisitando slot para a GPU para nossos programas *Vertex* e *Fragment Shaders*

In [6]:
# Request a program and shader slots from GPU
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)


### Compilando o *Vertex Shader*
Se há algum erro em nosso programa *Vertex Shader*, nosso app para por aqui.

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o *Fragment Shader*
Se há algum erro em nosso programa *Fragment Shader*, nosso app para por aqui.

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")


### Associando os programas compilados ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Make program the default program
glUseProgram(program)


# Definição de Elementos

### Preparando dados para enviar a GPU
Nesse momento, compilamos nossos *Vertex* e *Fragment Shaders* para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

In [12]:
def desenha_circulo(x_center, y_center, num_vertices, radius):
    circulo = np.zeros(num_vertices, [("position", np.float32, 2)])
    pi = np.pi
    angle = 0.0
    for counter in range(num_vertices):
        angle += 2*pi/num_vertices 
        x = math.cos(angle)*radius + x_center
        y = math.sin(angle)*radius + y_center
        circulo[counter] = [x,y]
    
    return circulo

def desenha_pa():
    pa = np.zeros(12, [("position", np.float32, 2)])
    pa['position'] = [
        (0.033, 0.5), (0.033, 0.513), (-0.033, 0.5), (-0.033, 0.513),
        (0.015, 0.1), (0.015, 0.5), (-0.015, 0.1), (-0.015, 0.5),
        (0.1, 0.1), (0.1, 0.0), (-0.1, 0.1), (-0.1, 0.0)
    ]
    pa = np.append(pa, desenha_circulo(0, 0, 32, 0.1))

    return pa

In [13]:
# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(154, [("position", np.float32, 2)])

# preenchendo as coordenadas de cada vértice
vertices['position'] = [
    # -------------- Flor 1 (0 - 21) --------------#
    # Meio
    (-0.01, +0.1), (+0.08, +0.02), (-0.08, 0), (+0.02, -0.07),
    # Petala 1
    (-0.01, +0.1), (+0.16, +0.29), (-0.13, +0.32),
    # Petala 2
    (+0.28, +0.29), (+0.04, +0.05), (+0.31, +0.07),
    # Petala 3
    (+0.33, -0.06), (+0.06, -0.03), (+0.2, -0.28),
    # Petala 4
    (+0.02, -0.07), (-0.07, -0.24), (+0.13, -0.31),
    # Petala 5
    (-0.05, -0.03), (-0.2, -0.25), (-0.34, -0.07),
    # Petala 6
    (-0.36, +0.06), (-0.05, +0.05), (-0.2, +0.2),

    # -------------- Flor 2 (22 - 70) --------------#
    # Meio
    (0, +0.1), (+0.08, +0.05), (+0.08, -0.04), (0, -0.09),
    (-0.08, -0.04), (-0.08, +0.05), (0, +0.1),
    # Petala 1
    (0, +0.1), (0, +0.19), (+0.08, +0.24), (+0.16, +0.19),
    (+0.16, +0.1), (+0.08, +0.05), (0, +0.1),
    # Petala 2
    (+0.08, +0.05), (+0.16, +0.1), (+0.24, +0.06), (+0.24, -0.04),
    (+0.16, -0.09), (+0.08, -0.04), (+0.08, +0.05),
    # Petala 3
    (+0.08, -0.04), (+0.16, -0.09), (+0.16, -0.18), (+0.08, -0.23),
    (0, -0.18), (0, -0.09), (+0.08, -0.04),
    # Petala 4
    (0, -0.09), (0, -0.18), (-0.08, -0.22), (-0.16, -0.18),
    (-0.16, -0.09), (-0.08, -0.04), (0, -0.09),
    # Petala 5
    (-0.08, -0.04), (-0.16, -0.09), (-0.24, -0.04), (-0.24, +0.05),
    (-0.16, +0.1), (-0.08, +0.05), (-0.08, -0.04),
    # Petala 6
    (-0.08, +0.05), (-0.16, +0.1), (-0.16, +0.19), (-0.08, +0.24), 
    (0, +0.19), (0, +0.1), (-0.08, +0.05),

    # -------------- Flor 3 (71 - 113) --------------#
    # Meio
    (0, +0.1), (+0.07, +0.06), (+0.07, -0.02), (0, -0.06), 
    (-0.07, -0.02), (-0.07, +0.06), (0, +0.1),
    # Petala 1
    (+0.01, +0.12), (+0.03, +0.2), (+0.11, +0.2),
    (+0.14, +0.13), (+0.08, 0.08), (+0.01, +0.12),
    # Petala 2
    (+0.09, +0.06), (+0.17, +0.09), (+0.21, +0.02),
    (+0.17, -0.04), (+0.09, -0.02), (+0.09, +0.06),
    # Petala 3
    (+0.09, -0.04), (+0.15, -0.09), (+0.12, -0.17),
    (+0.03, -0.16), (+0.01, -0.08), (+0.09, -0.04),
    # Petala 4
    (-0.01, -0.08), (-0.03, -0.17), (-0.11, -0.17),
    (-0.15, -0.1), (-0.08, -0.04), (-0.01, -0.08),
    # Petala 5
    (-0.09, -0.02), (-0.18, -0.05), (-0.22, +0.02),
    (-0.17, +0.09), (-0.09, +0.06), (-0.09, -0.02),
    # Petala 6
    (-0.09, +0.07), (-0.15, +0.13), (-0.11, +0.21),
    (-0.03, +0.2), (-0.01, 0.11), (-0.09, +0.07),

    # -------------- Canteiro (114 - 153) --------------#
    # Borda
    (-0.63, -0.63), (-0.63, 0.63), (0.63, -0.63), (0.63, 0.63),
    # Fundo
    (-0.6, -0.6), (-0.6, 0.6), (0.6, -0.6), (0.6, 0.6),
    # Padrão - 8 quadrados
    (-0.6, -0.6), (-0.6, -0.3), (-0.3, -0.6), (-0.3, -0.3),
    (-0.6, 0.0), (-0.6, 0.3), (-0.3, 0.0), (-0.3, 0.3),
    (-0.3, -0.3), (-0.3, 0.0), (0.0, -0.3), (0.0, 0.0),
    (-0.3, 0.3), (-0.3, 0.6), (0.0, 0.3), (0.0, 0.6),
    (0.0, -0.6), (0.0, -0.3), (0.3, -0.6), (0.3, -0.3),
    (0.0, 0.0), (0.0, 0.3), (0.3, 0.0), (0.3, 0.3),
    (0.3, -0.3), (0.3, 0.0), (0.6, -0.3), (0.6, 0.0),
    (0.3, 0.3), (0.3, 0.6), (0.6, 0.3), (0.6, 0.6),

]
    # -------------- Flor 4(154-313) --------------------#   

vertices = np.append(vertices,desenha_circulo(0.1, 0.1, 32 ,0.1))
vertices = np.append(vertices,desenha_circulo(0.1, -0.1, 32 ,0.1))
vertices = np.append(vertices,desenha_circulo(-0.1, -0.1, 32 ,0.1))
vertices = np.append(vertices,desenha_circulo(-0.1, 0.1, 32 ,0.1))
vertices = np.append(vertices,desenha_circulo(0, 0, 32 ,0.1))

    # -------------- Pá(314 - 357) --------------------#   
vertices = np.append(vertices,desenha_pa())





### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [14]:
# Adicionar os offsets aqui:
OFFSET_FLOR_1 = 0
OFFSET_FLOR_2 = 22
OFFSET_FLOR_3 = 71
OFFSET_CANTEIRO = 114
OFFSET_FLOR_4 = 154
OFFSET_PA = 314
#OFFSET_FLOR_5


In [15]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, enviamos todo o conteúdo da variável `vertices`.
Veja os parâmetros da função `glBufferData` [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [16]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Associando variáveis do programa `GLSL` (*Vertex Shaders*) com nossos dados
Primeiro, definimos o byte inicial e o offset dos dados.

In [17]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável `position` (que guarda coordenadas dos nossos vértices). Definimos essa variável no *Vertex Shader*.

In [18]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)


A partir da localização anterior, indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável `position` (aqui identificada na posição `loc`).

Outros parâmetros:
* Definimos que possui duas coordenadas.
* Que cada coordenada é do tipo `float` (`GL_FLOAT`).
* Que não se deve normalizar a coordenada (`False`).

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [19]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)


###  Aqui, pegamos a localização da variável color (`uniform`) para que possamos alterá-la em nosso laço da janela.

In [20]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0


### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [21]:
# exemplo para matriz de translacao
t_x = 0
t_y = 0
s_x = 1
s_y = 1
theta = 0
fixa = False

flor = 1


def key_event(window, key, scancode, action, mods):
    global t_x, t_y, s_x, s_y, theta, flor, fixa

    print('[key event] key=', key)
    # print('[key event] scancode=', scancode)
    # print('[key event] action=', action)
    # print('[key event] mods=', mods)

    # Transladar com as setas
    if key == 265:
        t_y += 0.01  # cima
    if key == 264:
        t_y -= 0.01  # baixo
    if key == 263:
        t_x -= 0.01  # esquerda
    if key == 262:
        t_x += 0.01  # direita
    if key == 77:
        s_x += 0.01  # aumenta
        s_y += 0.01
    if key == 78:
        s_x -= 0.01  # diminui
        s_y -= 0.01
    if key == 71:
        theta += 0.1
    if key == 72:
        theta -= 0.1
    if key == 257:
        fixa = True

    # Alterar entre as flores, as teclas 1, 2 e 3
    if key == 49:
        flor = 1
    elif key == 50:
        flor = 2
    elif key == 51:
        flor = 3
    elif key == 52:
        flor = 4


glfw.set_key_callback(window, key_event)


### Nesse momento, exibimos a janela.

In [22]:
glfw.show_window(window)


/home/flaviosalles/.local/lib/python3.10/site-packages/glfw/__init__.py:912: GLFWError: (65544) b'Wayland: Focusing a window requires user interaction'
  warnings.warn(message, GLFWError)


# Execução

### Objetos
Alguns objetos serão mostrados na janela, e para isso os definimos em diferentes funções.

In [23]:
def draw_flor1():
    offset = 0
    # Meio (Amarelo)
    glUniform4f(loc_color, 1, 1, 0, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset = offset + 4

    # Petalas (Brancas)
    for i in range(6):
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 3)
        offset = offset + 3

def draw_flor2():
    offset = 22
    glUniform4f(loc_color, 0.41, 0.35, 0.80, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
    offset = offset + 7

    for i in range(3):
        glUniform4f(loc_color, 0.28, 0.24, 0.54, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
        offset = offset + 7
        glUniform4f(loc_color, 1, 1, 1, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
        offset = offset + 7
    
def draw_flor3():
    offset = 71
    glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 7)
    offset = offset + 7

    for i in range(6):
        glUniform4f(loc_color, 0.87, 0.19, 0.39, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 6)
        offset = offset + 6

def draw_flor4():
    offset = 154
    for i in range(4):
        glUniform4f(loc_color, 0.87, 0.87, 0.0, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, offset, 32)
        offset = offset + 32

    glUniform4f(loc_color, 0.9, 0.9, 0.9, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)

def draw_pa():
    offset = OFFSET_PA
    glUniform4f(loc_color, 0.9, 0.9, 0.9, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset = offset + 4
    glUniform4f(loc_color, 0.9, 0.9, 0.9, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset = offset + 4
    glUniform4f(loc_color, 0.4, 0.4, 0.4, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset = offset + 4
    glUniform4f(loc_color, 0.4, 0.4, 0.4, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, offset, 32)
    offset = offset + 32




In [24]:
def draw_canteiro():
    offset = 114
    # Borda
    glUniform4f(loc_color, 0.20, 0.11, 0.03, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset = offset+4
    # Fundo
    glUniform4f(loc_color, 0.38, 0.22, 0.07, 1.0)
    glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
    offset = offset+4
    # Padrao
    for i in range(8):
        glUniform4f(loc_color, 0.38, 0.26, 0.07, 1.0)
        glDrawArrays(GL_TRIANGLE_STRIP, offset, 4)
        offset = offset+4

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a `OpenGL`.

A novidade agora é a função `glDrawArrays()`.

Tal função recebe o tipo de primitiva (`GL_TRIANGLES`), o índice inicial do array de vértices (mostraremos todos os três vértices, por isso começamos com $0$) e a quantidade de vértices (`len(vertices)`).

In [25]:
while not glfw.window_should_close(window):
    glfw.poll_events()

    # Definindo a cor de fundo
    glClearColor(0.45, 0.59, 0.23, 1.0)
    glClear(GL_COLOR_BUFFER_BIT)

    # Transformações Geométricas
    mat_fixa = np.array([       1.0, 0.0, 0.0, 0.0,
                                0.0, 1.0, 0.0, 0.0,
                                0.0, 0.0, 1.0, 0.0,
                                0.0, 0.0, 0.0, 1.0], np.float32)
    
    # mat_transformation = np.array( [math.cos(theta)*s_x, -math.sin(theta)*s_y, 0.0, -math.cos(theta)*s_x*t_x + math.sin(theta)*s_y*t_y + t_x,
    #                                 math.sin(theta)*s_x, math.cos(theta)*s_y, 0.0, -math.cos(theta)*s_y*t_y - math.sin(theta)*s_x*t_x + t_y,
    #                                 0.0, 0.0, 1.0, 0.0,
    #                                 0.0, 0.0, 0.0, 1.0], np.float32)
    mat_transformation = np.array( [math.cos(theta)*s_x, -math.sin(theta)*s_y, 0.0, t_x,
                                    math.sin(theta)*s_x, math.cos(theta)*s_y, 0.0, t_y,
                                    0.0, 0.0, 1.0, 0.0,
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_fixa)

    draw_canteiro()

    if flor == 1:
        loc = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
        draw_flor1()
        
    elif flor == 2:
        loc = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
        draw_flor2()
    elif flor == 3:
        loc = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
        draw_flor3()
    elif flor == 4:
        loc = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformation)
        draw_flor4()
    else:
        print("Flor não reconhecida")

    glfw.swap_buffers(window)

glfw.terminate()
